In [2]:
from model import predict_YOLO, predict_YOLO3_cphdata, predict_print_YOLO3_cphdata
from yolo3_one_file_to_detect_them_all import * 

import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array
from keras.backend import expand_dims
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from pathlib import Path

from PIL import Image
from os import listdir
from os.path import isfile, join


import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/oskarlassen/Desktop/DeepLearning/Project/Deep-Learning-Project/")

from CopActNet.tools import reader, writer
from CopActNet import config as cfg

ModuleNotFoundError: No module named 'model'

## Load the pretrained YOLO

In [5]:
# load yolov3 model
model = load_model('model.h5')

## Load pickle vid

In [6]:
pkl_data = reader.load_pickle("/Users/oskarlassen/Desktop/DeepLearning/Project/Deep-Learning-Project/Data/ny_video.pkl")

In [54]:
annotate_index = []


for i in range(pkl_data.shape[0]):
    
    ####### MAKING PICKLE IMAGE READY TO GO INTO YOLO
    img_pkl = pkl_data[i]
    # scale pixel values to [0, 1]
    img = img_pkl.astype('float32')
    img /= 255.0
    # add a dimension so that we have one sample
    img = expand_dims(img, 0)
    
    ####### RETURN ########

    labels = YOLO_pkl_predict(img, model)
    if ("bicycle" in labels) or ("person" in labels):
        annotate_index.append(1)
    else:
        annotate_index.append(0)
    
    if i % 200 == 0:
        print(i)

0
200
400
600
800
1000
1200
1400
1600


In [66]:
annotate_index_plus = []

val_prev = 0
for val in annotate_index:
    if val == 1:
        annotate_index_plus.append(1)
    elif val_prev == 1:
        annotate_index_plus.append(1)
    else:
        annotate_index_plus.append(0)
    val_prev = val

In [82]:
writer.pickle_obj(Path.joinpath(cfg.ROOT_DIR,"Frames_forannotation.pkl"),pkl_data[np.array(annotate_index_plus, dtype = bool)])